In [ ]:
# %% Deep learning - Section 18.165
#    The Conv2d class in PyTorch

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time
import sys
import imageio.v2          as imageio

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [ ]:
# %% When using PyTorch, convolutions will be most likely run via the class Conv2d

# Parameters
chans_in  = 3   # (RGB)
chans_out = 15  # (feature maps)
krn_size  = 5   # (odd; 15 kernels of size 5x5)
stride    = 1   # (no stride if =1)
padding   = 0   # (no padding if =0)

# Class instance
c = nn.ConvTranspose2d(chans_in,chans_out,krn_size,stride,padding)

print(c),print()

# Check weight tensor
print(f'Weights size: {c.weight.shape}')
print(f'Bias size: {c.bias.shape}')


In [ ]:
# What do these kernels look like?

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(3,5,figsize=(phi*5,5))

for i,ax in enumerate(axs.flatten()):
    ax.imshow(torch.squeeze(c.weight[0,i,:,:]).detach(),cmap='plasma')
    ax.set_title(f'L1(0) -> L2 ({i+1})')
    ax.axis('off')

plt.tight_layout()

plt.savefig('figure24_transpose_convolution.png')
plt.show()
files.download('figure24_transpose_convolution.png')


In [ ]:
# %% Convolve an image

# Image (N images, RGB, width, height)
img_size = (1,3,64,64)
img      = torch.rand(img_size)

# PyTorch wants channels first, but matplotlib wants channels last. So, tensors
# dims order must be permuted to visualize
img2view = img.permute(2,3,1,0).numpy()
print(img.shape)
print(img2view.shape)

plt.imshow(np.squeeze(img2view));

# convolve the image with the filter bank (set of 'chans_out' kernels)
conv_out = c(img)

print(img.shape)
print(conv_out.shape)


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(3,5,figsize=(phi*5,5))

for i,ax in enumerate(axs.flatten()):

    # extract feature map of the current convolution result
    I = torch.squeeze(conv_out[0,i,:,:]).detach()

    ax.imshow(I,cmap='plasma')
    ax.set_title(f'Conv. filter {i+1}')
    ax.axis('off')

plt.suptitle('Transpose convolving noise with noise')
plt.tight_layout()

plt.savefig('figure25_transpose_convolution.png')
plt.show()
files.download('figure25_transpose_convolution.png')
